In [2]:
import pandas as pd

Step 1: combine job_posting_details and job_description files

In [ ]:
jpd1 = pd.read_csv('/Users/apoltavets/anna-apps/annalyticai/careear-stats/output/job_posting_details/20250825-2005.tsv', sep='\t')
jpd2 = pd.read_csv('/Users/apoltavets/anna-apps/annalyticai/careear-stats/output/job_posting_details/20250825-2056.tsv', sep='\t')
jpd3 = pd.read_csv('/Users/apoltavets/anna-apps/annalyticai/careear-stats/output/job_posting_details/20250825-2059.tsv', sep='\t')

jpd = pd.concat([jpd1, jpd2, jpd3], ignore_index=True)
jpd.drop_duplicates(subset=['linkedin_job_id'], inplace=True)

In [ ]:
jd1 = pd.read_csv('/Users/apoltavets/anna-apps/annalyticai/careear-stats/output/job_description/20250825-2005.tsv', sep='\t')
jd2 = pd.read_csv('/Users/apoltavets/anna-apps/annalyticai/careear-stats/output/job_description/20250825-2056.tsv', sep='\t')
jd3 = pd.read_csv('/Users/apoltavets/anna-apps/annalyticai/careear-stats/output/job_description/20250825-2059.tsv', sep='\t')

jd = pd.concat([jd1, jd2, jd3], ignore_index=True)
jd.drop_duplicates(subset=['linkedin_job_id'], inplace=True)
jd = jd.drop(['job_title'], axis=1)
df = jpd.merge(jd, on='linkedin_job_id', how='inner')
df.to_csv('/Users/apoltavets/anna-apps/annalyticai/careear-stats/output/linkedin_jobs_20250825.csv', index=False)

Step 2: clean job titles

In [1]:
# Load the combined data
df = pd.read_csv('/Users/apoltavets/anna-apps/annalyticai/careear-stats/output/linkedin_jobs_20250825.csv')

NameError: name 'pd' is not defined

In [51]:
import re

def title_cleanup(job_title):
       if not isinstance(job_title, str):  # Check if the value is not a string
              return job_title

       tmp = re.sub(r'\(.*?\)', '', job_title).strip()
       tmp1 = re.sub(r'-\s*\d+[,\d]*/\w+\s+\w+$', '', tmp).strip()
       tmp2 = tmp1.replace('Sr.', 'Senior').replace('Sr ', 'Senior ')
       return tmp2

# Apply the function
df['job_title_upd'] = df['job_title'].apply(title_cleanup)


Step 3: filter out healthcare, education, finance, legal, construction, customer service jobs

In [ ]:
import re
titles = df["job_title"].drop_duplicates().to_list()
titles1 = []
for t in titles:
    titles1.append(re.sub(r'[^a-zA-Z\s]', '', str(t)))
    
keywords = (" ".join(titles1)).lower().split(" ")
kdf = pd.DataFrame(keywords, columns=['keyword'])

In [ ]:
kdf.groupby('keyword').size().sort_values(ascending=False)

In [163]:
df_filtered =(df[~df['organization_id'].isin([9185, 55381, 10914, 24034, 6755,
       # health organizations
       25522,
       10769,
       30412,
       14461,
       1340385,
       27573,
       33458365,
       1016,
       292629,
       11580,
       6250,
       265869,
       38521,
       42925,
       854537,
       166209,
       11819824,
       166821,
       8302,
       104175816
       ])]
)

In [164]:
df_filtered.info()

<class 'pandas.core.frame.DataFrame'>
Index: 11356 entries, 0 to 12061
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   linkedin_job_id     11356 non-null  int64  
 1   organization_name   11355 non-null  object 
 2   location            11356 non-null  object 
 3   job_title           11352 non-null  object 
 4   payer_name          11355 non-null  object 
 5   organization_id     11356 non-null  int64  
 6   job_post_date       11356 non-null  object 
 7   job_posting_millis  11356 non-null  float64
 8   job_description     11356 non-null  object 
dtypes: float64(1), int64(2), object(6)
memory usage: 887.2+ KB


In [166]:
conditions = (~(df['job_title'].str.lower().str.contains('nurse', na=False)) &
              (~df['job_title'].str.lower().str.contains('patient', na=False)) &
              (~df['job_title'].str.lower().str.contains('clinic', na=False)) &
              (~df['job_title'].str.lower().str.contains('therapist', na=False)) &
              (~df['job_title'].str.lower().str.contains('radiologic', na=False)) &
              (~df['job_title'].str.lower().str.contains('plebotomist', na=False)) &
              (~df['job_title'].str.lower().str.contains('veterinarian', na=False)) &
              (~df['job_title'].str.lower().str.contains('perfusionist', na=False)) &
              (~df['job_title'].str.lower().str.contains('ophtalmic', na=False)) &
              (~df['job_title'].str.lower().str.contains('surgery', na=False)) &
              (~df['job_title'].str.lower().str.contains('cardiology', na=False)) &
              (~df['job_title'].str.lower().str.contains('surgen', na=False)) &
              (~df['job_title'].str.lower().str.contains('hospital ', na=False)) &
              (~df['job_title'].str.lower().str.contains('hospital,', na=False)) &
              (~df['job_title'].str.lower().str.contains('controller', na=False)) &
              (~df['job_title'].str.lower().str.contains('tax', na=False)) &
              (~df['job_title'].str.lower().str.contains('accountant', na=False)) &
              (~df['job_title'].str.lower().str.contains('financial consultant', na=False)) &
              (~df['job_title'].str.lower().str.contains('administrative assistant', na=False)) &
              (~df['job_title'].str.lower().str.contains('teacher', na=False)) &
              (~df['job_title'].str.lower().str.contains('coordinator', na=False)) &
              (~df['job_title'].str.lower().str.contains('superintendent', na=False)) &
              (~df['job_title'].str.lower().str.contains('bridge', na=False)) &
              (~df['job_title'].str.lower().str.contains('estimator', na=False)) &
              (~df['job_title'].str.lower().str.contains('legal', na=False)) &
              (~df['job_title'].str.lower().str.contains('call center', na=False)) &
              (~df['job_title'].str.lower().str.contains('representative', na=False)) &
              (~df['job_title'].str.lower().str.contains('civil', na=False)) &
              (~df['job_title'].str.lower().str.contains('baker', na=False))
              )

df_filtered = df_filtered[conditions]

/var/folders/x8/ps_xddr95734lgrb6gqpr7g80000gp/T/ipykernel_60544/918165978.py:32: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_filtered = df_filtered[conditions]


In [171]:
dfu = (df_filtered[['linkedin_job_id', 'job_title', 'organization_name', 'job_description', 'payer_name', 'organization_id']].groupby(['job_title', 'organization_name', 'job_description', 'payer_name', 'organization_id'])
 .first()
.reset_index()
.sort_values(by=['job_title', 'organization_name']))

In [172]:
dfu.count().max()

np.int64(5834)

Lensa is a career site that helps job seekers find great jobs in the US. We are not a staffing firm or agency.
 Lensa does not hire directly for these jobs, but promotes jobs on LinkedIn on behalf ...

In [60]:
dfu = pd.read_csv('/Users/apoltavets/anna-apps/annalyticai/careear-stats/output/dfu_20250825_filtered.csv')

In [61]:
dfu['is_it'] = 1

In [64]:
df_dfu = df[['job_title', 'organization_name']].merge(dfu[['job_title', 'organization_name', 'is_it']], on=['job_title', 'organization_name'], how='left')

In [66]:
df_dfu['is_it'] = df_dfu['is_it'].fillna(0)

In [67]:
df_dfu['is_it'].value_counts()

is_it
1.0    16630
0.0     2179
Name: count, dtype: int64

In [71]:
df_dfu.to_csv('/Users/apoltavets/anna-apps/annalyticai/careear-stats/output/df_dfu_20250825_filtered.csv', index=False)

In [76]:
from joblib import load

def predict_it(job_title):
    X = [job_title]
    MODEL_PATH = "models/it_classifier_rf.joblib"
    ENCODER_PATH = "models/e5_encoder.joblib"
    
    # Use joblib to load the model and encoder
    clf = load(MODEL_PATH)
    enc = load(ENCODER_PATH)
    
    # Encode the input and make predictions
    X_encoded = enc.encode(X)
    y = clf.predict(X_encoded)
    return y[0]

In [78]:
df_dfu['predicted'] = predict_it(df_dfu.loc[0, 'job_title'])

RuntimeError: Attempting to deserialize object on a CUDA device but torch.cuda.is_available() is False. If you are running on a CPU-only machine, please use torch.load with map_location=torch.device('cpu') to map your storages to the CPU.

In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

df = pd.read_csv('/Users/apoltavets/anna-apps/annalyticai/careear-stats/data/job_titles_classified.csv')
df = df[df['title'].notna()]
X = df['title']  # Feature: job titles  
y = df['is_it']      # Target: is_it column  

# Split the data into training and testing sets  
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert job titles into numerical features using CountVectorizer  
vectorizer = CountVectorizer()
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

# Train a Random Forest Classifier  
model = RandomForestClassifier(random_state=42)
model.fit(X_train_vec, y_train)

# Make predictions  
y_pred = model.predict(X_test_vec)

# Evaluate the model  
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
print("\nClassification Report:\n", classification_report(y_test, y_pred))

# Example: Predicting for new job titles  
new_job_titles = ['Network Administrator', 'Marketing Specialist', 'Cloud Engineer']
new_job_titles_vec = vectorizer.transform(new_job_titles)
predictions = model.predict(new_job_titles_vec)

for title, prediction in zip(new_job_titles, predictions):
    print(f"Job Title: {title} => Predicted is_it: {prediction}")



Accuracy: 0.9506382978723404

Classification Report:
               precision    recall  f1-score   support

          No       0.91      1.00      0.95       538
         Yes       1.00      0.91      0.95       637

    accuracy                           0.95      1175
   macro avg       0.95      0.95      0.95      1175
weighted avg       0.95      0.95      0.95      1175

Job Title: Network Administrator => Predicted is_it: Yes
Job Title: Marketing Specialist => Predicted is_it: No
Job Title: Cloud Engineer => Predicted is_it: Yes


In [9]:
import pandas as pd
import os

# Specify the folder containing the CSV files  
folder_path = '/Users/apoltavets/anna-apps/annalyticai/careear-stats/output/job_posting_details/'
csv_files = [file for file in os.listdir(folder_path) if file.endswith('.tsv')] 
dataframes = [] 
for file in csv_files:
    file_path = os.path.join(folder_path, file)
    df = pd.read_csv(file_path, sep='\t')
    dataframes.append(df)
    
combined_df = pd.concat(dataframes, ignore_index=True)
print(combined_df.shape)
combined_df.drop_duplicates(subset=['linkedin_job_id'], inplace=True)
print(combined_df.shape)

/var/folders/x8/ps_xddr95734lgrb6gqpr7g80000gp/T/ipykernel_51498/2913625382.py:13: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  combined_df = pd.concat(dataframes, ignore_index=True)


(13956, 9)

In [22]:
combined_df = combined_df[combined_df['job_title'].notna()]
combined_df.drop_duplicates(subset=['organization_name', 'payer_name', 'job_title', 'job_description'], inplace=True)
import re

def title_cleanup(job_title):
    if not isinstance(job_title, str):  # Check if the value is not a string
        return job_title

    tmp = re.sub(r'\(.*?\)', '', job_title).strip()
    tmp1 = re.sub(r'-\s*\d+[,\d]*/\w+\s+\w+$', '', tmp).strip()
    tmp2 = tmp1.replace('Sr.', 'Senior').replace('Sr ', 'Senior ')
    return tmp2

# Apply the function
combined_df['job_title_upd'] = combined_df['job_title'].apply(title_cleanup)
combined_df = combined_df[combined_df['job_title_upd'].notna()]

/var/folders/x8/ps_xddr95734lgrb6gqpr7g80000gp/T/ipykernel_51498/4034364273.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  combined_df.drop_duplicates(subset=['organization_name', 'payer_name', 'job_title', 'job_description'], inplace=True)
/var/folders/x8/ps_xddr95734lgrb6gqpr7g80000gp/T/ipykernel_51498/4034364273.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  combined_df['job_title_upd'] = combined_df['job_title'].apply(title_cleanup)


In [24]:

vectorizer = load(MODEL_PATH)
model = load(ENCODER_PATH)

xvec = vectorizer.transform(combined_df['job_title_upd'])
xvec.shape
combined_df['is_it'] = model.predict(xvec)


(7661, 2883)

In [31]:
df_it = combined_df[combined_df['is_it'] == 'Yes']
df_it.shape

(4325, 11)

In [33]:
df_it['job_post_date'].value_counts()

job_post_date
2025-07-06    4063
2025-07-07     262
Name: count, dtype: int64

In [37]:
df = pd.read_csv('/Users/apoltavets/anna-apps/annalyticai/careear-stats/data/LinkedIn_Titles_Marked_20250829.csv')

In [38]:
df

,job_title_upd,organization_name,is_IT
0,IT Service Manager,Modern Office Methods,IT
1,IT Business Systems Administrator I,GreenState Credit Union,IT
2,Digital Print Manager,A Hiring Company,Non-IT
3,Senior Software Engineer,Enlighten,IT
4,Applied Integration and Power Systems Research...,National Renewable Energy Laboratory,IT
...,...,...,...
4102,Java / Cloud Solutions Architect/ W2 contract,Dice - JW,IT
4103,Site Reliability Engineer /w2 contract,Dice - JW,IT
4104,Java Full Stack Developer,Dice - JW,IT
4105,"Software Engineer, Inference",Anthropic,IT
